In [2]:
from pygraphblas import *
#from _pygraphblas import lib
import pygraphblas.descriptor
import csv
import sys
import logging
import glob
import operator
import sys
sys.path.append("..")
from loader.data_loader import DataLoader
from algorithms.search import naive_bfs_levels



In [3]:
# Setup logger
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)-5s %(message)s'))
log = logging.getLogger(__name__)
log.propagate = False
log.addHandler(handler)
log.setLevel(logging.INFO)

### Load data

In [4]:
data_dir = '../../csvs/o1k/'
data_format = 'csv'
loader = DataLoader(data_dir, data_format)

#vertices, mappings, matrices = loader.load_all_csvs()

person = loader.load_vertex('person')
comment = loader.load_vertex('comment')

replyOf = loader.load_edge('replyOf', comment, comment)
knows = loader.load_edge('knows', person, person)
hasCreator = loader.load_edge('hasCreator', comment, person)



### Queries

In [5]:
# Query 1
def shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, num_of_interactions, person1_id, person2_id):

    person1_id_remapped = person.index2id[person1_id]
    person2_id_remapped = person.index2id[person2_id] 

    hasCreatorTransposed = hasCreator.transpose()

    personA_to_comment2 = hasCreatorTransposed @ replyOf
    
    person_to_person = personA_to_comment2.mxm(hasCreator, mask=knows)
    
    person_to_person_filtered = person_to_person.select(lib.GxB_GT_THUNK, num_of_interactions)
    
    overlay_graph = person_to_person_filtered.pattern()
    if num_of_interactions == -1:
        overlay_graph = knows
        
    levels = naive_bfs_levels(overlay_graph, person1_id_remapped)
    
    
    result = levels[person2_id_remapped] - 1 # Get hop count
    
    return result
    

In [6]:
x = shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, 1, 786, 799)
print(f'RESULT: {x}', x==4)
x = shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, 1, 422, 736)
print(f'RESULT: {x}', x==-1)
x = shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, 1, 858, 587)
print(f'RESULT: {x}', x==4)
x = shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, -1, 266, 106)
print(f'RESULT: {x}', x==3)
x = shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, 0, 58, 402)
print(f'RESULT: {x}', x==3)
x = shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, 105, 608, 3) 
print(f'RESULT: {x}', x==-1)

RESULT: 3 False
RESULT: 4 False
RESULT: 4 True
RESULT: 3 True
RESULT: 3 True


NoValue: b''

In [45]:
# Optimized version: do not create overlay graph but investigate investigate KNOWS edges on-the-fly.
def step_frontier(frontier, seen, num_of_interactions, numpersons, hasCreator, replyOf, knows):
    frontierPersonIndices = frontier.to_lists()[0]
    
    if num_of_interactions >= 0:
        sel = Matrix.from_lists(frontierPersonIndices, frontierPersonIndices, [1]*len(frontierPersonIndices), numpersons, numpersons)
        K1 = sel.mxm(hasCreator.transpose()).mxm(replyOf            ).mxm(hasCreator, mask=knows).select(lib.GxB_GT_THUNK, num_of_interactions)
        K2 = sel.mxm(hasCreator.transpose()).mxm(replyOf.transpose()).mxm(hasCreator, mask=knows).select(lib.GxB_GT_THUNK, num_of_interactions)
        K = K1*K2
    else:
        K = knows

    push = False
    if (push):
        next = frontier.vxm(K, mask=seen, desc=descriptor.ooco)
    else:
        # K is not symmetric --> we need to transpose it
        next = K.transpose().mxv(frontier, mask=seen, desc=descriptor.ooco)

    return (next, seen)

def shortest_distance_over_frequent_communication_paths_opt(person, replyOf, knows, hasCreator, num_of_interactions, person1_id, person2_id):
    person1_id_remapped = person.index2id[person1_id]
    person2_id_remapped = person.index2id[person2_id] 

    numpersons = len(person.id2index)
    frontier1 = Vector.from_lists([person1_id_remapped], [True], numpersons)
    frontier2 = Vector.from_lists([person2_id_remapped], [True], numpersons)
    seen1 = frontier1
    seen2 = frontier2

    for level in range(1, numpersons//2):
        #print("===== " + str(level) + " =====")
        #print("frontier persons: " + str(frontierPersonIndices))

        # frontier 1
        next1, seen1 = step_frontier(frontier1, seen1, num_of_interactions, numpersons, hasCreator, replyOf, knows)

        # has frontier1 intersected frontier2's previous state?
        intersection1 = next1 * seen2
        if intersection1.nvals > 0:
            return level*2-1
        # emptied the component of person1
        if next1.nvals == 0:
            return -1

        # frontier 2
        next2, seen2 = step_frontier(frontier2, seen2, num_of_interactions, numpersons, hasCreator, replyOf, knows)

        # do frontier1 and frontier2's current states intersect?
        intersection2 = next1 * next2
        if intersection2.nvals > 0:
            return level*2
        # emptied the component of person2
        if next2.nvals == 0:
            return -1

        # step the frontiers
        seen1 = seen1 + next1
        frontier1 = next1

        seen2 = seen2 + next2
        frontier2 = next2

x = shortest_distance_over_frequent_communication_paths_opt(person, replyOf, knows, hasCreator, 1, 786, 799)
print(f'RESULT: {x}', x==4)
x = shortest_distance_over_frequent_communication_paths_opt(person, replyOf, knows, hasCreator, 1, 422, 736)
print(f'RESULT: {x}', x==-1)
x = shortest_distance_over_frequent_communication_paths_opt(person, replyOf, knows, hasCreator, 1, 858, 587)
print(f'RESULT: {x}', x==4)
x = shortest_distance_over_frequent_communication_paths_opt(person, replyOf, knows, hasCreator, -1, 266, 106)
print(f'RESULT: {x}', x==3)
x = shortest_distance_over_frequent_communication_paths_opt(person, replyOf, knows, hasCreator, 0, 58, 402)
print(f'RESULT: {x}', x==3)
x = shortest_distance_over_frequent_communication_paths_opt(person, replyOf, knows, hasCreator, 105, 608, 3) 
print(f'RESULT: {x}', x==-1)


RESULT: 4 True
RESULT: -1 True
RESULT: 4 True
RESULT: 3 True
RESULT: 3 True
RESULT: -1 True
